In [62]:
!unzip testing_zip_file

unzip:  cannot find or open testing_zip_file, testing_zip_file.zip or testing_zip_file.ZIP.


In [1]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b8df5ca81ad187b2a69d9c88415916aebd835d5bd3873f4409bb3f797bbe7029
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, QuestionAnsweringPipeline
from transformers import AlbertTokenizer, AlbertForQuestionAnswering, AlbertConfig, PreTrainedTokenizer
from tqdm.auto import tqdm
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval

In [6]:
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.


"""dict_theme_ind[<THEME>]['<start|end>']"""
df_paras = pd.read_csv("input_paragraph_small10.csv", index_col=0)
para_startInd = json.loads(pd.read_csv("./theme_interval_small10.csv").to_json(orient="records"))


theme_para_dict = {}
dict_theme_ind = {}
dict_paras = df_paras.to_dict()
for i in range(1,len(dict_paras['theme'].keys())+1):
  if dict_paras['theme'][i] in theme_para_dict:
    theme_para_dict[dict_paras['theme'][i]].append(dict_paras['paragraph'][i])
  else:
    theme_para_dict[dict_paras['theme'][i]] = [(dict_paras['paragraph'][i])]
    dict_theme_ind[dict_paras['theme'][i]] = {"start": i}

# Dummy code.
# paragraph_ds = []
# global_model = {}


In [7]:
retriver_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
passage_enc_dict = {}
for theme in tqdm(theme_para_dict):
  passage_enc_dict[theme] = retriver_model.encode(theme_para_dict[theme])
  

100%|██████████| 10/10 [11:18<00:00, 67.80s/it]


In [9]:
import pickle
with open("passage_enc_dict.pkl", "rb") as f:
  passage_enc_dict = pickle.load(f)

In [18]:
# Allowed to make changes.
tokenizer = AlbertTokenizer.from_pretrained("twmkn9/albert-base-v2-squad2")
model = AlbertForQuestionAnswering.from_pretrained("twmkn9/albert-base-v2-squad2")
model = model.to(device)
def get_theme_model(theme):
  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.
  return tokenizer, model

In [40]:
inputs = tokenizer("who is it di die do it?", "I dont know what happend.", return_tensors="pt")
inputs['input_ids'].size()

torch.Size([1, 18])

In [53]:
# Allowed to make changes.


def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"]

  for question in tqdm(questions):
    #-------------------------------------   
    # add your prediction methodology here.
    #-------------------------------------
    ans = {}
    ans["question_id"] = question["id"]

    ########### predict paragraph id ########

    id = (passage_enc_dict[theme]@retriver_model.encode(question['question']).reshape((-1,1))).flatten().argmax()
    para_id = dict_theme_ind[theme]["start"] + id
    ans["paragraph_id"] = para_id
    ######### predict answer ###########
    text = theme_para_dict[theme][id]
    
    inputs = tokenizer(question["question"], text, return_tensors="pt")
    if((inputs['input_ids'].size()[1])>500):
      ans["paragraph_id"] = -1
      ans["answers"] = " "
      continue
    inputss = inputs.to(device)

    
    with torch.no_grad():
        outputs = model(**inputss)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    ans["answers"] = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
    if(ans["answers"] == ''):
      ans["paragraph_id"] = -1
      ans["answers"] = " "
    # print(question['question'])
    # print(ans)
    pred_out.append(ans)

In [54]:
# NOT allowed to make changes. 

# All theme prediction.
questions = json.loads(pd.read_csv("input_question_small10.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("theme_interval_small10.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in tqdm(theme_intervals):
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('output_prediction_small10.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/523 [00:00<?, ?it/s]

  0%|          | 0/490 [00:00<?, ?it/s]

  0%|          | 0/193 [00:00<?, ?it/s]

  0%|          | 0/249 [00:00<?, ?it/s]

...

  0%|          | 0/563 [00:00<?, ?it/s]

  0%|          | 0/166 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

In [55]:
# NOT allowed to make changes. 

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [56]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("output_prediction_small10.csv")
truth = pd.read_csv("ground_truth_small10.csv")

truth.paragraph_id = [ [x] for x in truth['paragraph_id'].to_list()]
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("input_question_small10.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1


In [59]:
theme_weights = {}
for theme_interval in theme_intervals:
  theme_weights[theme_interval['theme']] = (- int(theme_interval["start"]) + int(theme_interval["end"]))/3382


In [60]:
theme_weights

{'Beyoncé': 0.1543465405085748,
 'Frédéric_Chopin': 0.14458900059136606,
 'Sino-Tibetan_relations_during_the_Ming_dynasty': 0.05677114133648729,
 'The_Legend_of_Zelda:_Twilight_Princess': 0.07332939089296274,
 'Spectre_(2015_film)': 0.09432288586635128,
 'New_York_City': 0.166173861620343,
 'To_Kill_a_Mockingbird': 0.04878769958604376,
 'Solar_energy': 0.051744529863985804,
 'Kanye_West': 0.08249556475458308,
 'Buddhism': 0.12448255470136015}

In [61]:
# NOT allowed to make changes. 

# Final score.
inf_time_threshold = 200.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
# Weight would stay hidden from teams.
# theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
print (final_para_score)
print (final_qa_score)


0.7354049673200335
0.656198595866289
